<a href="https://colab.research.google.com/github/ethansong206/Climate-Plus-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages for the code

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

Load the Duke Dining csv file and make the data easier to handle. This includes but is not limited to:

1) Renaming locations

2) Changing the date from a string to a date object



In [2]:
DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv', 
                             dtype = {'Priority 1': str, 'Priority 2': str,
                                      'Priority 3': str})
#print(DiningDataFull['Unit Name'].value_counts()) # Show how many entries there are of each location

#below code to make 'Unit Name' column easier to handle
#can add more lines given more locations
def location_rename(location):
    if("Marketplace" in location): #Combine data for Marketplace Kitchen and Marketplace Special Event
        return "Marketplace"
    if("Marine Lab" in location):
        return "DuML"
    if("Trinity" in location):
        return "Trinity"
    if("Freeman" in location):
        return "Freeman"
    return None

DiningDataFull['Unit Name'] = DiningDataFull.apply(lambda d: location_rename(d['Unit Name']), axis = 1)

DiningDataFull['Purchase Date'] = DiningDataFull.apply(lambda d: datetime.strptime(d['Purchase Date'], '%m/%d/%Y').date(), axis = 1)

print("Rows in DiningDataFull: ", DiningDataFull.shape[0])

Rows in DiningDataFull:  90534


#**\*\*\*WIP\*\*\***

Extract the unit names for each item into a new column called `Unit`, then rename each unit into a more simple label (i.e. LB CS to LB). Convert cans and bottle cases into the equivalent value in OZ. Make a new column called `Total Food Amount` for the total amount of food in the current unit of measurement, before converting to grams. Make a new column called `Unit(g)` for converting all units to the equivalent value in grams, then filter out main descriptor word(s) in `Vendor Item Description` into a new column called `Food Name`.

---
Note on Exclusion: Some units are left out in this first calculation of emissions as they are either not directly food (i.e. gloves) or are too difficult to go through individually and find a measurement that is not ambiguous. The total number of entries left out is 12171. **The number of non-food items in this amount can be calculated later.**

---
Note on Conversions: The column `Vendor Item Purchase Unit` is in the format x/y n which should be read as x bags of y units of n food.

---

Take the column with units in grams and multiply by the `Receive Quantity` if provided into a new column called `Total Grams`. If there is no value in `Receive Quantity`, then assume it is the value 1.


In [122]:
#this code extracts just the unit information
DiningDataFull['Unit'] = DiningDataFull['Vendor Item Purchase Unit'].str.extract(r" ?([A-Za-zÀ-ÿ ]*)$").astype(str)
#below code to simplify redundant labels (i.e. LB CS to LB)
def unit_rename(unit):
    if('LB' in unit) | ('lb' in unit) | ('Lb' in unit) | ('Pound' in unit):
        return 'LB'
    if('OZ' in unit) | ('oz' in unit) | ('Oz' in unit) | (' Z' in unit):
        return 'OZ'
    if('GA' in unit) | ('Gal' in unit):
        return 'GA'
    if('QT' in unit):
        return 'QT'
    if('PT' in unit) | ('Pint' in unit) | ('PINT' in unit):
        return 'PT'
    if('LT' in unit):
        return 'LT' 
    if('BU' in unit) | ('Bushel' in unit): #bushels
        return 'BU'
    if('KG' in unit):
        return 'KG' 
    if('GR' in unit): #grams
        return 'GR' 
    if('ML' in unit) | ('ml' in unit) | ('Ml' in unit): #milliliters
        return 'ML'
    if('CN' in unit) | ('Can' in unit):
        return 'CN'
    if('Bottle Case' in unit):
        return 'Bottle Case'
    return None
#Notes: Bottle Case is 64 oz each, find # of can and translate to oz, anything with EA is not included for now (~9000)
DiningDataFull['Unit'] = DiningDataFull.apply(lambda d: unit_rename(d['Unit']), axis = 1)

#make a new dataset for just entries with known units
DDReduced = DiningDataFull[DiningDataFull['Unit'].notna()]
#print(DDReduced.head())
print("Excluded rows: ", DiningDataFull.shape[0] - DDReduced.shape[0])


#CONVERT UNITS
#first convert cans to OZ
#using estimates for weight of cans through https://food.unl.edu/article/how-interpret-can-size-numbers
DDReduced = DDReduced.replace({'#10 CN' : '110.5 OZ'}, regex = True)
DDReduced = DDReduced.replace({'#10 Can' : '110.5 OZ'}, regex = True)
DDReduced = DDReduced.replace({'#300 CN' : '15 OZ'}, regex = True)
DDReduced = DDReduced.replace({'CN' : 'OZ'})
#next convert bottle case to OZ
DDReduced = DDReduced.replace({' Bottle Case' : '/64 OZ'}, regex = True)
DDReduced = DDReduced.replace({'Bottle Case' : 'OZ'})

#find total amount of food before converting to grams
#DO THIS NEXT
DDReduced['Total Food Amount'] = DDReduced['Vendor Item Purchase Unit'].str.extract(r'^[a-zA-Z]* ?-?/? ?([0-9]*/?[0-9.]*-?[0-9.]*)')
DDReduced['Range'] = DDReduced['Total Food Amount'].str.extract(r'([0-9.]*-[0-9.]*)')
DDReduced['Range'] = DDReduced['Range'].astype(str)
DDReduced['Range'] = DDReduced['Range'].replace({'nan' : '0'})
DDReduced['Range Average'] = DDReduced['Range'].replace({'-' : '+'}, regex = True)
DDReduced['Range Average'] = DDReduced.apply(lambda d: eval(d['Range Average']), axis = 1)
DDReduced['Range Average'] /= 2

DDReduced.loc[has_range, 'Total Food Amount'] = (
    DDReduced.loc[has_range, 'Total Food Amount'].str.split('/').str[0]
    + '/'
    + DDReduced.loc[has_range, 'Range Average'].astype(str)
)

DDReduced = DDReduced.drop(['Range', 'Range Average'], axis = 1)

DDReduced['Total Food Amount'] = DDReduced['Total Food Amount'].replace({'' : '1'})
DDReduced['Receive Quantity'] = DDReduced['Receive Quantity'].fillna(1)
DDReduced['Total Food Amount'] = DDReduced['Total Food Amount'].replace({'/' : '*'}, regex = True)
DDReduced['Total Food Amount'] = DDReduced.apply(lambda d: eval(d['Total Food Amount']), axis = 1)
DDReduced['Total Food Amount'] = DDReduced['Total Food Amount'].astype(float) * DDReduced['Receive Quantity'].astype(float)

#index = 0
#for row in DDReduced['Total Food Amount']:
#  if row == 0:
#    print(row)
#    print(index)
#  index += 1

#print(DDReduced['Total Food Amount'][74450:74500])


print(DDReduced.head())

Excluded rows:  12126
         Unit Name    Vendor Purchase Date Vendor Item ID  \
86255  Marketplace  US Foods    2023-03-28         185306   

                                 Vendor Item Description  \
86255  TEA BAG, ICED BLACK ORANGE PEKOE CAFFEINATED 3...   

      Vendor Item Purchase Unit                   Product Group    Brand  \
86255                32/3 OZ CS  651701 Beverages Non-Alcoholic  RITUALS   

       Receive Quantity Priority 1 Priority 2 Priority 3 Unit  \
86255               0.0        NaN        NaN        NaN   OZ   

       Total Food Amount  
86255                0.0  
     Unit Name    Vendor Purchase Date Vendor Item ID  \
0  Marketplace  US Foods    2019-07-01        5018999   
1  Marketplace  US Foods    2019-07-01        5400692   
2  Marketplace  US Foods    2019-07-01        8012643   
3  Marketplace  US Foods    2019-07-01         731349   
4  Marketplace  US Foods    2019-07-01          68106   

                             Vendor Item Description 

Making one dataset for each location and each year

In [4]:
location_col = DDReduced['Unit Name']
DiningData_Marketplace = DDReduced[location_col == "Marketplace"]
DiningData_DuML = DDReduced[location_col == "DuML"]
DiningData_Trinity = DDReduced[location_col == "Trinity"]
DiningData_Freeman = DDReduced[location_col == "Freeman"]

date_col = DDReduced['Purchase Date']
DiningData_2019 = DDReduced[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
DiningData_2020 = DDReduced[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
DiningData_2021 = DDReduced[(date_col >= pd.Timestamp(2021, 1, 1)) & (date_col < pd.Timestamp(2022, 1, 1))]
DiningData_2022 = DDReduced[(date_col >= pd.Timestamp(2022, 1, 1)) & (date_col < pd.Timestamp(2023, 1, 1))]
DiningData_2023 = DDReduced[(date_col >= pd.Timestamp(2023, 1, 1)) & (date_col < pd.Timestamp(2024, 1, 1))]

<ipython-input-4-51466aaa99e5>:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2019 = DiningDataReduced[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
<ipython-input-4-51466aaa99e5>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2020 = DiningDataReduced[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
<ipython-input-4-51466aaa99e5>:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-c